### Alice möchte Bob 10000 satoshis senden welche Bob anschließend wieder ausgibt. Bob gibt Alice dabei verschiedene Möglichkeiten wie die Satoshis für ihn gelocked werden.

Die ersten Möglichkeit ist das Versenden der Satoshis an einen öffentlichen Schlüssel welcher durch einen von Bob kontrollierten privaten Schlüssel erzeugt wurde. Dieser Fall wird **P2PK-Transaktion** genannt.

In [1]:
import sys
sys.path.insert(0, "/media/henning/Volume/Programming/Bitcoin/bitcoin/test/functional")
from test_framework.test_shell import TestShell
from io import BytesIO
from decimal import Decimal

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2pk_script, p2ms_script
from src.ecdsa import PrivateKey

### Aufgrund der Tatsache, dass wir mit einer "leeren" Blockchain beginnen, müssen zunächst ein paar Vorarbeiten geleistet werden.
    1. Verbindung zur Blockchain in regtest Modus aufbauen
    2. Anlegen zweier Nodes und Wallets (Alice und Bob)
    3. Mining 101 Blöcke damit Alice Wallet Coins enthält
    4. Funding der Transaktion welche später als Input für die P2PK-Locking Transaktion genutzt wird.
    5. Bob erzeugt seinen öffentlichen Schlüssel an den die Funds gebunden werden sollen

In [2]:
test = TestShell().setup(num_nodes=2, setup_clean_chain=True)
alice = test.nodes[0]
bob = test.nodes[1]

alice.createwallet("alice", descriptors=False)
bob.createwallet("bob", descriptors=False)

alice_address_0 = alice.getnewaddress("", "legacy")
test.generatetoaddress(alice, 101, alice_address_0)

alice_address_1 = alice.getnewaddress("", "legacy")
alice_p2pk_funding_tx = alice.sendtoaddress(alice_address_1, 0.00011)
alice_p2pk_funding_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2pk_funding_tx)["hex"])
test.generatetoaddress(alice, 1, alice_address_0)

bob_address_1 = bob.getnewaddress("", "legacy")
bob_private_key_wif_format = bob.dumpprivkey(bob_address_1)
bob_private_key = PrivateKey.convert_wif_format(bob_private_key_wif_format)
bob_private_key_int_format = bob_private_key.get_private_key_int()
bob_public_key = bob_private_key.get_public_key()



2024-02-29T12:27:09.388000Z TestFramework (INFO): PRNG seed is: 5423618138543387768
2024-02-29T12:27:09.390000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_3_o1rt2q


**1.** In der ersten Transaktion werden von Alice die 10000 Satoshis an den von Bob kontrollierten öffentlichen Schlüssel versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen. 

In [3]:
alice_private_key_wif_format = alice.dumpprivkey(alice_address_1)
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

alice_previous_tx_id_to_spent = alice_p2pk_funding_tx_parsed["txid"]

for tx_out in alice_p2pk_funding_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00011000"):
        alice_previous_tx_index_to_spent = tx_out["n"]
        alice_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
alice_amount_to_spent = 10000

**1.1** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's öffentlichen Schlüssel zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: <Bob's public Key> + OP_CHECKSIG generieren.

In [4]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2pk_script(bob_public_key.sec_format())
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2PK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice unsignierte P2PK Lockingtransaktion: 0100000001db314da5d0c52337106d43b87df5fe00b8f2cde15a9c1deb58ce6954b3158f310000000000ffffffff011027000000000000232103897505002112d4f8c280ae1fc448c9c5f5cfdced54dd85be94ba17067c95c4acacffffffff


**1.1.1** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [5]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, [alice_private_key_int_format], alice_script_sig)

print(f"Alice signierte P2PK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

alice_p2pk_locking_tx = alice.sendrawtransaction(alice_locking_transaction.serialize_transaction().hex())
test.generatetoaddress(alice, 1, alice_address_0)

print(alice_p2pk_locking_tx)

Alice signierte P2PK Lockingtransaktion: 0100000001db314da5d0c52337106d43b87df5fe00b8f2cde15a9c1deb58ce6954b3158f31000000006a47304402206c9df40ee2fd65380b5a7a6d27799b0105713efad4e4cc73936602b508acff3a02200c4e72cc02352f73c631be0a62ef9bae0d90608ca39559122c62628dd4dd600b0121033155292335b14e3c7aa2e46ab9495a0c1dccf69b5e781e4aa903ba26b19385a8ffffffff011027000000000000232103897505002112d4f8c280ae1fc448c9c5f5cfdced54dd85be94ba17067c95c4acacffffffff
c57adf89e472e7f50f57ccee0c4865385eb830570b010fa19ca4073fad192967


**2.** Nun möchte Bob die an seinen öffentlichen Schlüssel gesendeten Satoshis weiter versenden. Dazu muss Bob im Inputfeld (in dem auf den eben von Alice erzeugten Output referenziert wird), zum Beweis das er  berechtigt ist aen Output auszugeben lediglich eine Signatur mit Bob's privatem Schlüssel stehen. Damit Bob das notwendige Transaktionsformular ausfüllen kann benötigt er folgende Informationen.

In [6]:
alice_p2pk_locking_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2pk_locking_tx)["hex"])

bob_previous_tx_id_to_spent = alice_p2pk_locking_tx_parsed["txid"]

#for tx_out in alice_p2pk_locking_tx_parsed["vout"]:
#    if tx_out["value"] == Decimal("0.00010000"):
#        bob_previous_tx_index_to_spent = tx_out["n"]
#        bob_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]
bob_previous_script_pub_key_to_spent = alice_script_pubkey.serialize_script().hex()
#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [7]:
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script())

script_pubkey = p2pk_script(alice_public_key.sec_format())
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Bob unsignierte P2PK Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

Bob unsignierte P2PK Spendingtransaktion: 0100000001672919ad3f07a49ca10f010b5730b85e3865480ceecc570ff5e772e489df7ac50000000000ffffffff0128230000000000002321033155292335b14e3c7aa2e46ab9495a0c1dccf69b5e781e4aa903ba26b19385a8acffffffff


**2.2** Nun kann Bob diese Transaktion mit seinem privaten Schlüssle signieren und versenden.

In [8]:
bob_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(bob_previous_script_pub_key_to_spent)//2)[2:]}{bob_previous_script_pub_key_to_spent}")))
bob_spending_transaction.sign_transaction(0, [bob_private_key_int_format], bob_script_sig)

print(f"Bob signierte P2PK Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

bob_p2pk_spending_tx = bob.sendrawtransaction(bob_spending_transaction.serialize_transaction().hex())

print(bob_p2pk_spending_tx)

TestShell().shutdown()

Bob signierte P2PK Spendingtransaktion: 0100000001672919ad3f07a49ca10f010b5730b85e3865480ceecc570ff5e772e489df7ac500000000494830450221009927b9b2557e70304983c0b351b791538deb679c8ea13b1ed265d2abf409e91d02204667994c47928c02073331cd76563c1f07830226ee3b7a27d9d62e0bec6e645a01ffffffff0128230000000000002321033155292335b14e3c7aa2e46ab9495a0c1dccf69b5e781e4aa903ba26b19385a8acffffffff
590e90d303a0a1bfe8191b980d5ff8ec849d9bf46b6a02b9d5bb7d6d7c36665c
2024-02-29T12:27:11.393000Z TestFramework (INFO): Stopping nodes
2024-02-29T12:27:11.495000Z TestFramework (INFO): Cleaning up /tmp/bitcoin_func_test_3_o1rt2q on exit
2024-02-29T12:27:11.495000Z TestFramework (INFO): Tests successful


### Bob hat einen Freund Namens Charlie. Bob möchte das die Coins welche er von Alice geschickt bekommt NUR ausgegeben werden können wenn Bob und Charlie die Transaktion mit ihren jeweils eigenen privaten Schlüsseln signieren. Alice muss daher die Coins nicht an den öffentlichen Schlüssel von Bob schicken sondern an ein Skript welches die beiden öffentlichen Schlüssel von Bob und Charlie enthält. Diese Art der Transaktion bezeichen wir als **P2MS** oder **Bare Multisig**

### Aufgrund der Tatsache, dass wir mit einer "leeren" Blockchain beginnen, müssen zunächst ein paar Vorarbeiten geleistet werden.
    1. Verbindung zur Blockchain in regtest Modus aufbauen
    2. Anlegen dreier Nodes und Wallets (Alice, Bob und Charlie)
    3. Mining 101 Blöcke damit Alice Wallet Coins enthält
    4. Funding der Transaktion welche später als Input für die P2PK-Locking Transaktion genutzt wird.
    5. Bob erzeugt seinen öffentlichen Schlüssel an den die Funds gebunden werden sollen
    6. Charlie erzeugt seinen öffentlichen Schlüssel an den die Funds gebunden werden sollen

In [9]:
test = TestShell().setup(num_nodes=3, setup_clean_chain=True)
alice = test.nodes[0]
bob = test.nodes[1]
charlie = test.nodes[2]

alice.createwallet("alice", descriptors=False)
bob.createwallet("bob", descriptors=False)
charlie.createwallet("charlie", descriptors=False)

alice_address_0 = alice.getnewaddress("", "legacy")
test.generatetoaddress(alice, 101, alice_address_0)

alice_address_1 = alice.getnewaddress("", "legacy")
alice_p2pk_funding_tx = alice.sendtoaddress(alice_address_1, 0.00011)
alice_p2pk_funding_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2pk_funding_tx)["hex"])
test.generatetoaddress(alice, 1, alice_address_0)

bob_address_1 = bob.getnewaddress("", "legacy")
bob_private_key_wif_format = bob.dumpprivkey(bob_address_1)
bob_private_key = PrivateKey.convert_wif_format(bob_private_key_wif_format)
bob_private_key_int_format = bob_private_key.get_private_key_int()
bob_public_key = bob_private_key.get_public_key()

charlie_address_1 = charlie.getnewaddress("", "legacy")
charlie_private_key_wif_format = charlie.dumpprivkey(charlie_address_1)
charlie_private_key = PrivateKey.convert_wif_format(charlie_private_key_wif_format)
charlie_private_key_int_format = charlie_private_key.get_private_key_int()
charlie_public_key = charlie_private_key.get_public_key()

2024-02-29T12:27:11.506000Z TestFramework (INFO): PRNG seed is: 1655237743351615362
2024-02-29T12:27:11.507000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_3_o1rt2q


**1.** In der ersten Transaktion werden von Alice die 10000 Satoshis an den von Bob und den von Charlie kontrollierten öffentlichen Schlüssel versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen. 

In [10]:
alice_private_key_wif_format = alice.dumpprivkey(alice_address_1)
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

alice_previous_tx_id_to_spent = alice_p2pk_funding_tx_parsed["txid"]

for tx_out in alice_p2pk_funding_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00011000"):
        alice_previous_tx_index_to_spent = tx_out["n"]
        alice_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
alice_amount_to_spent = 10000

**1.1** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's öffentlichen Schlüssel zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: <Bob's public Key> + OP_CHECKSIG generieren.

In [11]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2ms_script([bob_public_key.sec_format(), charlie_public_key.sec_format()], 2, 2)
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2MS Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice unsignierte P2MS Lockingtransaktion: 01000000010c387d45f489d042752666063f292bba26251554b195e0fb1ad1b7fc01827af60000000000ffffffff01102700000000000047522103a06e6991879b5a5415d8792feb10e0e8bac565e64e9e427cc62e9c6ee31efca92103febab0dfe1cad324f4344d0c4fe2055fdd1acf2992c44bd497abd62b7b08d34a52aeffffffff


**1.1.1** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [12]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, [alice_private_key_int_format], alice_script_sig)

print(f"Alice signierte P2MS Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

alice_p2ms_locking_tx = alice.sendrawtransaction(alice_locking_transaction.serialize_transaction().hex())
test.generatetoaddress(alice, 1, alice_address_0)

print(alice_p2ms_locking_tx)

Alice signierte P2MS Lockingtransaktion: 01000000010c387d45f489d042752666063f292bba26251554b195e0fb1ad1b7fc01827af6000000006b483045022100f46d43cd6b6785c75d185ff489e3234139e6734d930c597f44a5fde18e34f61602202d3d17bdab560a14834083388e66f650c5ab15beb57412584a6557b944a4d54c0121025c1e1024ec9a797ef08f30e8c5fd54ae0e06c17546d13fd413ab7ac0fdefa574ffffffff01102700000000000047522103a06e6991879b5a5415d8792feb10e0e8bac565e64e9e427cc62e9c6ee31efca92103febab0dfe1cad324f4344d0c4fe2055fdd1acf2992c44bd497abd62b7b08d34a52aeffffffff
0a3f326c77014fb2a324bfa09d30b44b0ce69c210fe7a452626c29186c8e8317


**2.** Nun möchten Bob und Charlie die an ihre beiden öffentlichen Schlüssel gebundenen Satoshis weiter versenden. Dazu müssen die beiden im Inputfeld (in dem auf den eben von Alice erzeugten Output referenziert wird), zum Beweis das sie berechtigt sind den Output auszugeben jeweils eine Signatur mit ihrem privatem Schlüssel vorlegen. Die Reihenfolge in der die Signaturen vorliegen muss die selbe Reihenfolge sein in der die öffentlichen Schlüssel im P2MS Skript vorliegen. Damit Bob das notwendige Transaktionsformular ausfüllen kann benötigt er folgende Informationen.

In [13]:
alice_p2ms_locking_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2ms_locking_tx)["hex"])

bob_previous_tx_id_to_spent = alice_p2ms_locking_tx_parsed["txid"]

for tx_out in alice_p2ms_locking_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00010000"):
        bob_previous_tx_index_to_spent = tx_out["n"]
        bob_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [14]:
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script())

script_pubkey = p2pk_script(alice_public_key.sec_format())
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Bob unsignierte P2MS Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

Bob unsignierte P2MS Spendingtransaktion: 010000000117838e6c18296c6252a4e70f219ce60c4bb4309da0bf24a3b24f01776c323f0a0000000000ffffffff0128230000000000002321025c1e1024ec9a797ef08f30e8c5fd54ae0e06c17546d13fd413ab7ac0fdefa574acffffffff


**2.2** Nun können Bob und Charlie diese Transaktion mit ihren privaten Schlüssle signieren und versenden.

In [15]:
bob_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(bob_previous_script_pub_key_to_spent)//2)[2:]}{bob_previous_script_pub_key_to_spent}")))
bob_spending_transaction.sign_transaction(0, [bob_private_key_int_format, charlie_private_key_int_format], bob_script_sig, 2, 2)

print(f"Bob signierte P2MS Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

bob_p2ms_spending_tx = bob.sendrawtransaction(bob_spending_transaction.serialize_transaction().hex())

print(bob_p2ms_spending_tx)

TestShell().shutdown()

Bob signierte P2MS Spendingtransaktion: 010000000117838e6c18296c6252a4e70f219ce60c4bb4309da0bf24a3b24f01776c323f0a00000000920047304402203208b1dcd1db140eb67d31c9830908ea77d3ee47f13f3192ac2cc2fd20cc3fb40220079af87f1bfcb1bbd8769f832e2365ce40fd4d9076d550635d75b143d8ced65a01483045022100a492b157c139acdea1074fced17ab2b729f7e2a53676462b1856590685567a420220259102b9144975e7bebd499befde6e85905e6bd12444dde8cedd99c5f4caf8e401ffffffff0128230000000000002321025c1e1024ec9a797ef08f30e8c5fd54ae0e06c17546d13fd413ab7ac0fdefa574acffffffff
b1318f0b0538d5c9e45b91cc70b67b83abeed5ee3078b9592a058feff6ce1f49
2024-02-29T12:27:12.775000Z TestFramework (INFO): Stopping nodes
2024-02-29T12:27:12.878000Z TestFramework (INFO): Cleaning up /tmp/bitcoin_func_test_3_o1rt2q on exit
2024-02-29T12:27:12.879000Z TestFramework (INFO): Tests successful
